In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import dask.array as da
# import netCDF4
import pandas as pd
# from datetime import datetime
# from datetime import timedelta
# from scipy import stats
# import scipy.interpolate as interp
from dask.distributed import Client, Variable
client = Client()
%matplotlib inline

In [2]:
from functions import ccplot, binning

In [3]:
import seaborn as sns
# Use seaborn style defaults and set default figure size
plt.style.use('seaborn-pastel')
sns.set_theme(style="ticks")

In [4]:
# Colormap selection
xr.set_options(cmap_divergent='RdBu_r', cmap_sequential='GrBu')

In [5]:
mfdata_DIR = './data/GPM_data/*.nc'

ds = xr.open_mfdataset(mfdata_DIR, chunks={'time':2000}, engine='netcdf4', combine='nested', concat_dim='time', parallel=True)

In [8]:
ds = ds.drop_dims('bnds')

In [10]:
ds = ds.transpose('time', 'lat', 'lon')

In [11]:
ds

<xarray.Dataset>
Dimensions:             (time: 369600, lon: 401, lat: 401)
Coordinates:
  * time                (time) datetime64[ns] 2000-06-01 ... 2021-06-30T23:30:00
  * lon                 (lon) float32 60.05 60.15 60.25 ... 99.85 99.95 100.1
  * lat                 (lat) float32 -0.05 0.05 0.15 0.25 ... 39.75 39.85 39.95
Data variables:
    precipitationCal    (time, lat, lon) float32 dask.array<chunksize=(2000, 401, 401), meta=np.ndarray>
    precipitationUncal  (time, lat, lon) float32 dask.array<chunksize=(2000, 401, 401), meta=np.ndarray>
    IRprecipitation     (time, lat, lon) float32 dask.array<chunksize=(2000, 401, 401), meta=np.ndarray>
Attributes:
    CDI:                             Climate Data Interface version 1.9.10 (h...
    Conventions:                     CF-1.6
    FileHeader:                      DOI=10.5067/GPM/IMERG/3B-HH/06;\nDOIauth...
    FileInfo:                        DataFormatVersion=6a;\nTKCodeBuildVersio...
    Grid.GridHeader:                 BinMethod=ARITHMETIC_MEAN;\nRegistration...
    Grid.fullnamepath:               /Grid
    DODS_EXTRA.Unlimited_Dimension:  time
    history:                         Tue Oct 26 15:21:29 2021: cdo mergetime ...
    CDO:                             Climate Data Operators version 1.9.10 (h...

In [13]:
mfdata_DIR2 = './data/era_data/*.nc'

ds2 = xr.open_mfdataset(mfdata_DIR2, chunks={'time':2000}, engine='netcdf4', combine='nested', concat_dim='time', parallel=True)

In [14]:
ds2

,Array,Chunk
Bytes,36.86 GiB,395.52 MiB
Shape,"(190879, 161, 161, 2)","(2000, 161, 161, 2)"
Count,454 Tasks,109 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,36.86 GiB,395.52 MiB
Shape,"(190879, 161, 161, 2)","(2000, 161, 161, 2)"
Count,454 Tasks,109 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
